<a href="https://colab.research.google.com/github/zhukuixi/Kaggle/blob/main/Kaggle_PGS_jan2021_optunaParameterTuningForXGboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Download data of "tabular-playground-series-jan-2021" from Kaggle

This is a regression competition with small amount of features:  

The training set is (300000,14)  
The test set is (200000,14)    
The cost function is RMSE.

In [2]:
import json
token = {"username":"kuixizhu","key":"2c3bcc7cb22446cf0dc296bd69fe3dc6"}
with open('/content/kaggle.json', 'w') as file:
  json.dump(token, file)
  

!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Assign path
!kaggle config set -n path -v /content
# Download data
!kaggle competitions download -c tabular-playground-series-jan-2021
# unzip data
!unzip /content/competitions/tabular-playground-series-jan-2021/tabular-playground-series-jan-2021.zip -d /content/competitions/tabular-playground-series-jan-2021/
# remove zip file
!rm /content/competitions/tabular-playground-series-jan-2021/tabular-playground-series-jan-2021.zip

- path is now set to: /content
100% 62.8M/62.8M [00:03<00:00, 23.2MB/s]
100% 62.8M/62.8M [00:03<00:00, 16.6MB/s]
Archive:  /content/competitions/tabular-playground-series-jan-2021/tabular-playground-series-jan-2021.zip
  inflating: /content/competitions/tabular-playground-series-jan-2021/sample_submission.csv  
  inflating: /content/competitions/tabular-playground-series-jan-2021/test.csv  
  inflating: /content/competitions/tabular-playground-series-jan-2021/train.csv  


In [ ]:
!pip install optuna
!pip install catboost

# Load in the ml package and data


In [4]:
# load in basic package
import os
import numpy as np
import pandas as pd

# load in ML package
import optuna
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

projectFolder = '/content/competitions/tabular-playground-series-jan-2021'

train = pd.read_csv(os.path.join(projectFolder,'train.csv'))
test  = pd.read_csv(os.path.join(projectFolder,'test.csv'))
sub = pd.read_csv(os.path.join(projectFolder,'sample_submission.csv'))

In [5]:
print(train.shape)
train.head()


(300000, 16)


,id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
0,1,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
1,3,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331
2,4,0.834950,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047,7.776091
3,5,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528,6.957716
4,8,0.935278,0.421235,0.303801,0.880214,0.665610,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772,7.951046


In [6]:
featureNames = [col for col in train.columns.to_list() if col not in ['id','target']]
X_train= train[featureNames]
Y_train= train['target']

# Build the optimization function using optuna 

In [7]:
# trial : the number of executions it will take
def objective(trial,data=X_train,target=Y_train):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 10000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': trial.suggest_categorical('random_state', [2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = xgb.XGBRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

# Show the Optimization Result and Visualization from different perspective

In [ ]:
study.trials_dataframe()

In [10]:
#plot_optimization_histor: shows the scores from all trials as well as the best score so far at each point.
optuna.visualization.plot_optimization_history(study)

In [11]:
#plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

In [12]:
'''plot_slice: shows the evolution of the search. You can see where in the hyperparameter space your search
went and which parts of the space were explored more.'''
optuna.visualization.plot_slice(study)

In [13]:
#plot_contour: plots parameter interactions on an interactive chart. You can choose which hyperparameters you would like to explore.
optuna.visualization.plot_contour(study, params=['alpha',
                            #'max_depth',
                            'lambda',
                            'subsample',
                            'learning_rate',
                            'subsample'])

In [14]:
#Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

In [15]:
#Visualize empirical distribution function
optuna.visualization.plot_edf(study)

Use the Best Hyperparameter of Xgboost to do prediction on Test set

In [16]:
Best_trial = study.best_trial.params
Best_trial["n_estimators"], Best_trial["tree_method"] = 10000, 'gpu_hist'
Best_trial

{'lambda': 0.07864582157862672,
 'alpha': 0.004989471976219203,
 'colsample_bytree': 0.7,
 'subsample': 0.6,
 'learning_rate': 0.008,
 'max_depth': 15,
 'random_state': 2020,
 'min_child_weight': 145,
 'n_estimators': 10000,
 'tree_method': 'gpu_hist'}

In [ ]:
preds = np.zeros(test.shape[0])
kf = KFold(n_splits=5,random_state=48,shuffle=True)
rmse=[]  # list contains rmse for each fold
n=0
for trn_idx, test_idx in kf.split(train[featureNames],train['target']):
    X_tr,X_val=train[featureNames].iloc[trn_idx],train[featureNames].iloc[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
    model = xgb.XGBRegressor(**Best_trial)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    preds+=model.predict(test[featureNames])/kf.n_splits
    rmse.append(mean_squared_error(y_val, model.predict(X_val), squared=False))
    print(f"fold: {n+1} ==> rmse: {rmse[n]}")
    n+=1

In [22]:
sub['target']=preds
sub.to_csv('submission.csv', index=False)